In [2]:
import os
from dotenv import load_dotenv
HOME = os.getenv('HOME')
load_dotenv(f'{HOME}/.zshrc')

from utils.db import ToricCY
from utils import format
db = ToricCY()

In [3]:
invols = db.find_invols({'H11': 3})

In [528]:
def format_invol(invol):
    pairs = []
    for sub_invol in invol.strip('{}').split(','):
        x, y = sub_invol.split('->')
        x = int(x.lstrip('D')) - 1
        y = int(y.lstrip('D')) - 1
        pair = sorted((x, y))
        if pair not in pairs:
            pairs.append(pair)
    return pairs
a = format_invol(ex['INVOL'])
a

[[2, 4], [3, 5]]

In [533]:
a = Permutation(a)

In [534]:
a.cyclic_form

[[2, 4], [3, 5]]

In [259]:
from utils.calc import get_srideal
# ex = invols[65]
# ex = invols[126]
ex = invols[127]
rescws = format.mat(ex['RESCWS'])
itensxd = format.mat(ex['ITENSXD'])
nverts = format.mat(ex['NVERTS'])
dresverts = format.mat(ex['DRESVERTS'])
k, n = dresverts.shape
triang = format.mat(ex['TRIANG'])
srideal = get_srideal(triang)
invol = format.invol(ex['INVOL'], k)

In [501]:
def get_srideal(R, triang):
    """Compute the Stanley-Reisner ideal"""
    k = R.ngens()
    n = len(triang[0])
    srideal = []
    for i in range(n):
        for c in map(set, combinations(range(k), i + 1)):
            for prev_c in srideal:
                if prev_c.issubset(c):
                    break
            else:
                for t in map(set, triang):
                    if c.issubset(t):
                        break
                else:
                    srideal.append(c)
    return srideal
a = get_srideal(R, triang)

In [485]:
import numpy as np
from itertools import permutations, combinations
from sympy.combinatorics.permutations import Permutation

from sage.rings.rational_field import QQ
from sage.geometry.polyhedron.constructor import Polyhedron
from sage.rings.polynomial.multi_polynomial_ideal import MPolynomialIdeal


def build_ring(k):
    """Build polynomial ring in k variables over the rational numbers"""
    R = QQ[tuple(f'x_{i}' for i in range(k))]
    return R


def compute_monomials(R, nverts, dresverts):
    """Eq. A.8 from arXiv:1411.1418"""
    X = np.array(R.gens())
    Delta = Polyhedron(vertices=nverts)
    npoints = np.array(Delta.integral_points())
    P_expon = (npoints @ dresverts.T) + 1
    P_monoms = np.power(X, P_expon).prod(axis=1)
    return P_monoms


def compute_srideal(R, triang):
    """
    Compute the Stanley-Reisner ideal, i.e. the sets of 
    divisors that never intersect
    """
    k = R.ngens()
    n = len(triang[0])
    srideal = []
    for i in range(n):
        for c in map(set, combinations(range(k), i + 1)):
            # SR ideal generators must be minimal
            for prev_c in srideal:
                if prev_c.issubset(c):
                    break
            else:
                for t in map(set, triang):
                    # 1-cones in a simplex correspond to intersecting divisors
                    if c.issubset(t):
                        break
                else:
                    srideal.append(c)
    X = R.gens()
    srideal = [R.ideal(*[X[i] for i in s]) for s in srideal]
    # The intersection of ideals == union of varieties
    # This ideal sweeps out all points that must be removed from the ambient space
    return MPolynomialIdeal.intersection(*srideal)


def compute_dim(I):
    """Compute the orientifold plane dimension of an ideal"""
    codim = I.dimension()
    dim = I.ring().krull_dimension() - codim
    return 3 + 2 * (3 - dim)


def compute_fixed_loci(R, P_monoms, invol, srideal):
    """Compute the loci on the CY3 that are fixed under the involution"""
    X = np.array(R.gens())
    X_sub = dict(zip(X, X[invol.array_form]))
    # Compare each monomial, so that coefficients remain arbitrary
    P_diff = np.vectorize(lambda x: x - x.subs(X_sub))(P_monoms)
    fixed_loci = R.ideal(*P_diff).minimal_associated_primes()
    # Remove points in the SR ideal from the fixed loci
    fixed_loci_reduced = []
    dimensions = []
    for locus_ideal in fixed_loci:
        locus_ideal = locus_ideal.quotient(srideal)
        if locus_ideal.dimension() > -1: # Quotient is trivial if dimension = -1
            fixed_loci_reduced.append(locus_ideal.gens())
            dimensions.append(compute_dim(locus_ideal))
    return dimensions, fixed_loci_reduced


def is_smooth(R, P_monoms, fixed_loci, srideal):
    X = np.array(R.gens())
    srideal_intersect = MPolynomialIdeal.intersection(*[R.ideal(*X[s]) for s in srideal])
    J_P_monoms = [x.jacobian_ideal() for x in P_monoms]
    trans_ideal = R.ideal(*P_monoms) + sum(J_P_monoms) + sum(fixed_loci)
    trans_ideal = trans_ideal.radical().quotient(srideal_intersect)
    return trans_ideal.dimension() < 0


def vform_terms(R, rescws):
    X = np.array(R.gens())
    k, m = rescws.shape
    n = k - m
    terms = {}
    Vp_factors = rescws * X.reshape(-1, 1)
    for p in permutations(range(k), k - n):
        wp = tuple(set(range(k)) - set(p))
        signp = Permutation(p + wp).signature()
        Vp = signp * Vp_factors[p, range(3)].prod()
        if wp in terms:
            terms[wp] += Vp
        else:
            terms[wp] = Vp
    return {k: v for k, v in terms.items() if v != 0}


def get_vform_parity(R, rescws, invol):
    X = np.array(R.gens())
    vform = vform_terms(R, rescws)
    vform_invol = vform_terms(R, rescws[invol.array_form])
    if set(vform) != set(vform_invol):
        return 0
    signs = [vform_invol[key] / vform[key] for key in vform]
    if all(x == invol.signature() for x in signs):
        return 1
    elif all(x == -1 * invol.signature() for x in signs):
        return -1
    else:
        return 0


# def complete_permutation(p, k):
#     return Permutation(p + tuple(i for i in range(k) if i not in p))


# def get_vform_parity(rescws, X):
#     k, m = rescws.shape
#     n = k - m
#     vform = 0
#     vform_invol = 0
#     for p in permutations(range(k), k - n):
#         vform_sign = complete_permutation(p, k).signature()
#         vform_coeff = rescws[p, range(3)].prod()
#         vform_monom = X[list(p)].prod()
#         vform += vform_sign * vform_coeff * vform_monom

#         p_invol = tuple(invol(i) for i in p)
#         vform_invol_sign = invol.signature() * complete_permutation(p_invol, k).signature()
#         vform_invol_coeff = rescws[p, range(3)].prod()
#         vform_invol_monom = X[list(p_invol)].prod()
#         vform_invol += vform_invol_sign * vform_invol_coeff * vform_invol_monom
#     parity = vform / vform_invol
#     if (parity == 1) or (parity == -1):
#         return parity
#     return 0


In [486]:
from tqdm import tqdm
all_elems = []
for ex in tqdm(invols, total=len(invols)):
    polyid = ex['POLYID']
    geomn = ex['GEOMN']
    triangn = ex['TRIANGN']
    involn = ex['INVOLN']
    rescws = format.mat(ex['RESCWS'])
    k = len(rescws)
    nverts = format.mat(ex['NVERTS'])
    dresverts = format.mat(ex['DRESVERTS'])
    triang = format.mat(ex['TRIANG'])
    srideal = get_srideal(triang)
    invol = format.invol(ex['INVOL'], k)

    R = build_ring(invol)
    P_monoms = get_polynomial(R, nverts, dresverts)
    fixed_loci = get_fixed_loci(R, P_monoms, invol, srideal)
    oplanes = [{'OIDEAL': x.gens(), 'ODIM': get_odim(x)} for x in fixed_loci]
    smooth = is_smooth(R, P_monoms, fixed_loci, srideal)
    vform_parity = get_vform_parity(R, rescws, invol)
    elem = {'POLYID': polyid, 'GEOMN': geomn, 'TRIANGN': triangn, 'INVOLN': involn, 'OPLANES': oplanes, 'SMOOTH': smooth, 'VOLFORMPARITY': vform_parity}
    all_elems.append(elem)

100%|██████████| 183/183 [02:51<00:00,  1.07it/s]


In [487]:
import pickle
with open('h11_3_test.pkl', 'wb') as fh:
    pickle.dump(all_elems, fh)

In [297]:
a = [x for x in all_elems if x['VOLFORMPARITY'] == -1][1]
a

{'POLYID': 249,
 'GEOMN': 1,
 'TRIANGN': 2,
 'INVOLN': 2,
 'OPLANES': [{'OIDEAL': [x_4, x_1], 'ODIM': 5},
  {'OIDEAL': [x_0*x_3 - x_2*x_5], 'ODIM': 7}],
 'SMOOTH': True,
 'VOLFORMPARITY': -1}

In [302]:
b = [x for x in invols if all(x[y] == a[y] for y in ['POLYID', 'GEOMN', 'TRIANGN', 'INVOLN'])][0]

In [303]:
b['SRIDEAL']

'{D1*D6,D3*D4,D2*D5*D7}'

In [304]:
b['INVOL']

'{D1->D3,D3->D1,D4->D6,D6->D4}'

In [305]:
b['RESCWS']

'{{0,0,1},{0,1,0},{1,0,0},{1,0,0},{0,1,0},{0,0,1},{2,2,2}}'

In [367]:
for i in range(len(invols)):
    if all_elems[i]['VOLFORMPARITY'] == -1:
        print(invols[i]['OPLANES'])
        print(all_elems[i]['OPLANES'])
        print(invols[i]['SRIDEAL'])
        print('-----')

[{'OIDEAL': ['x3^2*x4-x5^2*x6'], 'ODIM': 7}, {'OIDEAL': ['x1', 'x2', 'x3^2*x4+x5^2*x6'], 'ODIM': 3}]
[{'OIDEAL': [x_5, x_1, x_0], 'ODIM': 3}, {'OIDEAL': [x_2^2*x_3 - x_4^2*x_5], 'ODIM': 7}, {'OIDEAL': [x_3, x_1, x_0], 'ODIM': 3}, {'OIDEAL': [x_1, x_0, x_2^2*x_3 + x_4^2*x_5], 'ODIM': 3}]
{D3*D5,D3*D6,D4*D5,D1*D2*D4*D7,D1*D2*D6*D7}
-----
[{'OIDEAL': ['x1*x4-x3*x6'], 'ODIM': 7}]
[{'OIDEAL': [x_4, x_1], 'ODIM': 5}, {'OIDEAL': [x_0*x_3 - x_2*x_5], 'ODIM': 7}]
{D1*D6,D3*D4,D2*D5*D7}
-----
[{'OIDEAL': ['x1*x3-x2*x4'], 'ODIM': 7}]
[{'OIDEAL': [x_6], 'ODIM': 7}, {'OIDEAL': [x_0*x_2 - x_1*x_3], 'ODIM': 7}]
{D1*D4,D2*D3,D5*D6*D7}
-----
[{'OIDEAL': ['x1*x4-x3*x6'], 'ODIM': 7}]
[{'OIDEAL': [x_6, x_4, x_0*x_3 + x_2*x_5], 'ODIM': 3}, {'OIDEAL': [x_6, x_1, x_0*x_3 + x_2*x_5], 'ODIM': 3}, {'OIDEAL': [x_0*x_3 - x_2*x_5], 'ODIM': 7}]
{D2*D5,D3*D4,D1*D6*D7}
-----
[{'OIDEAL': ['x1*x2-x3*x4'], 'ODIM': 7}]
[{'OIDEAL': [x_6], 'ODIM': 7}, {'OIDEAL': [x_5, x_4, x_0*x_1 + x_2*x_3], 'ODIM': 3}, {'OIDEAL': [x_0*x_

In [368]:
[all_elems[i]['SMOOTH'] == invols[i]['SMOOTH'] for i in range(len(invols))]

[True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True

In [488]:
[all_elems[i]['VOLFORMPARITY'] == invols[i]['VOLFORMPARITY'] for i in range(len(invols))]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [489]:
[[x['ODIM'] for x in all_elems[i]['OPLANES']] for i in range(len(invols)) if all_elems[i]['VOLFORMPARITY'] == -1]

[[3, 7, 3, 3],
 [5, 7],
 [7, 7],
 [3, 3, 7],
 [7, 3, 7],
 [7],
 [7],
 [3, 7],
 [3, 3, 7],
 [7, 3, 7],
 [7, 5],
 [7],
 [3, 7],
 [7, 5],
 [7],
 [7, 7],
 [7],
 [3, 3, 3, 7],
 [7],
 [7, 3],
 [5, 7],
 [7, 5],
 [5, 7],
 [7],
 [7],
 [5, 7],
 [7, 7, 3],
 [3, 3, 7, 5],
 [3, 3, 5, 7],
 [7],
 [7],
 [7],
 [3, 7, 5],
 [7, 7, 7, 3],
 [3, 3, 7],
 [7],
 [7],
 [7, 5],
 [7],
 [3, 3, 7],
 [7],
 [7, 5],
 [3, 7],
 [7],
 [7],
 [7],
 [3, 3, 7],
 [7],
 [3, 3, 7],
 [7, 5],
 [7],
 [7],
 [3, 3, 7],
 [7, 5],
 [7],
 [5, 7],
 [5, 7],
 [3, 3, 7, 5],
 [5, 7],
 [7],
 [7],
 [7],
 [7],
 [7],
 [5, 7],
 [3, 5, 3, 7],
 [7],
 [3, 3, 7],
 [3, 3, 7],
 [5, 7],
 [7],
 [7],
 [7, 3],
 [3, 5, 3, 7],
 [3, 3, 7],
 [3, 7, 5],
 [3, 3, 7],
 [3, 7, 5],
 [7],
 [7],
 [3, 3, 7],
 [7, 5],
 [3, 7, 3],
 [7],
 [5, 7],
 [7],
 [7],
 [7],
 [3, 3, 7],
 [7],
 [7, 5],
 [7, 5],
 [3, 7],
 [7],
 [7],
 [3, 3, 7],
 [3, 3, 5, 7],
 [7, 3],
 [7, 3, 3]]